In [2]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import requests
import urllib.request, json 
import re
import time
from datetime import date

import hashlib

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()
    
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
    
def download_pdf(url,path,filename):
    try:
        
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup
def download_pdf_unmasked(url,path,filename):
    
        
    filename = path+filename

    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)

    return convert(filename)

    
def download_docx(book_link, book_name):
    the_book = requests.get(book_link, stream=True)
    with open(book_name, 'wb') as f:
      for chunk in the_book.iter_content(1024 * 1024 * 2):  # 2 MB chunks
        f.write(chunk)

In [4]:
BASE = "https://www.vermontjudiciary.org"
source = "https://www.vermontjudiciary.org/forms-library" 

today = date.today().strftime("%Y-%m-%d")
form_dest = "PDFS/VT_forms/All_forms"

soup = connect_to_page(source)

group_tag = soup.find("section",id="block-forms-library-facet-unit")

list_of_groups = group_tag.find_all('li', class_= "facet-item")

In [5]:
urls_per_cat = []
groups = []
pages_per_group = [8,3,2,18,2,8,2]
page_extension = "&search_api_fulltext=&page={}".format(0)


for i in list_of_groups:
    href = i.find('a',href=True)['href']
    groups.append(i.find('span', class_="facet-item__value").text)
    urls_per_cat.append(BASE+href)

print(urls_per_cat)
print(groups)
print(pages_per_group)

['https://www.vermontjudiciary.org/forms-library?f%5B0%5D=unit%3A1', 'https://www.vermontjudiciary.org/forms-library?f%5B0%5D=unit%3A2', 'https://www.vermontjudiciary.org/forms-library?f%5B0%5D=unit%3A3', 'https://www.vermontjudiciary.org/forms-library?f%5B0%5D=unit%3A4', 'https://www.vermontjudiciary.org/forms-library?f%5B0%5D=unit%3A5', 'https://www.vermontjudiciary.org/forms-library?f%5B0%5D=unit%3A6', 'https://www.vermontjudiciary.org/forms-library?f%5B0%5D=unit%3A7']
['Civil', 'Criminal', 'Environmental', 'Family', 'Judicial Bureau', 'Probate', 'Supreme Court']
[8, 3, 2, 18, 2, 8, 2]


In [10]:
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"])
pdfs = []
jur = 'VT'
source = "https://www.vermontjudiciary.org/forms-library" 
form_dest = "All_forms/"

for i in range(len(urls_per_cat)):

    for k in range(pages_per_group[i]):
        time.sleep(1)
        soup = connect_to_page(urls_per_cat[i]+"&search_api_fulltext=&page={}".format(k))

        for link in soup.find_all('a',href=True):
            if '.pdf' in link['href']:
                filename = re.sub("%20","_",link['href'].split('/')[-1])
                if filename not in pdfs:
                    group = groups[i]
                    title = link.text
                    url = link['href']
                    print(group)
                    print(title)
                    print(filename)
                    file_id = hashme(url)
                    download_pdf_unmasked(url,form_dest,file_id+".pdf")
                    files_df = files_df.append(pd.DataFrame([[file_id,jur,source,title,group,url,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]))
        print('##################')
        print('PAGE',k)
        print('##################')
        print(groups[i])
        

files_df.to_csv("VT_metaData.csv",index=False, encoding="utf-8")

Civil
Acceptance of Service
600-00028.pdf
Civil
ADA Request Customer
ADA_request_customer.pdf
Civil
Affidavit in Support of Request for Order Against Stalking or Sexual Assault
100-00248_Affidavit.pdf
Civil
Alternate Dispute Resolution Report
100-00294_0.pdf
Civil
Answer
100-00051.pdf
Civil
Appendix A - Eviction Certification 
Appendix_A_1.pdf
Civil
Appendix B - Foreclosure Certification
Appendix_B.pdf
Civil
Application to Perform 16.3 Mediation
Form_292_0.pdf
Civil
Application to Waive Filing Fees and Service Costs
600-00228_Application_to_Waive_Filing_Fees_and_Service_Costs.pdf
Civil
ARABIC Affidavit in Support of Request for Order Against Stalking or Sexual Assault 100-00248 
ARABIC_Affidavit_in_Support_of_Request_for_Order_Against_Stalking_or_Sexual_Assault_100-00248_.pdf
Civil
ARABIC Application to Waive Filing Fees and Service Costs 600-00228 
ARABIC_Application_to_Waive_Filing_Fees_and_Service_Costs_600-00228_.pdf
Civil
ARABIC Complaint for Order Against Stalking On Behalf Of A 